# Aprendizaje Automático - Tutorial de programación
Vamos a seguir el proceso de aprendizaje automático utilizando como vehículo las redes de neuronas. Vamos a ver unas cuantas librerías relevantas y cómo se utilizan en el contexto del aprendizaje automático.

# Extracción y tratamiento de datos
El conjunto de datos Iris es un ejercicio común y sencillo para un perceptrón multicapa. El problema consiste en, dadas cuatro variables numéricas, encontrar la especie de la flor que representan dichas variables. Es un problema de **clasificación** y lo vamos a tratar como tal. Nuestro primer objetivo es descargar los datos de https://archive.ics.uci.edu/ml/machine-learning-databases
, pre-procesarlos y después particionarlos en tres conjuntos: Entrenamiento (80%), validacion (10%) y test (10%).

In [1]:
# Descarga de datos
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data

--2022-10-26 18:11:25--  https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4551 (4.4K) [application/x-httpd-php]
Saving to: ‘iris.data’

iris.data           100%[===================>]   4.44K  --.-KB/s    in 0s      

2022-10-26 18:11:26 (85.1 MB/s) - ‘iris.data’ saved [4551/4551]



## Pandas

Pandas permite la gestión de datos en formato de tabla. En particular podemos leer los datos rapidamente con pd.read_csv

El resultado: una tabla de 5 columnas y un índice. Descubrimos que hay 150 elementos que utilizaremos.

In [2]:
import pandas as pd

df = pd.read_csv('iris.data', header=None)
df.columns = [f'var{x}' for x in range(4)] + ['class']
df

,var0,var1,var2,var3,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


# Sklearn
Sklearn contiene muchas utilidades para Machine Learning. Vamos a dar un ejemplo con dos de ellas: StandardScaler (normalización de media 0 y desviación 1) y train_test_split.

También convertimos la clase en una variable numérica usando las utilidades de pandas.

In [3]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=.2, random_state=42) # Extraer un 80% para train y 20% para validacion y test
valid, test = train_test_split(test, test_size=.5, random_state=42) # Extraer el 10% de validacion y el 10% de test
train = train.sample(frac=1.) # Desordenamos las filas en entrenamiento

In [4]:
ss = StandardScaler()

# Cada columna tiene ahora de media 0 y desviacion 1.
# El resultado es un array de numpy de dimensiones (120, 4)
train_x = ss.fit_transform(train.drop('class', axis=1)) 
valid_x = ss.transform(valid.drop('class', axis=1))
test_x = ss.transform(test.drop('class', axis=1))

In [5]:
# Queremos las clases en formato numerico
# Para cada elemento de la tabla encontramos su respectivo indice
#   en la lista classes.
classes = df['class'].unique().tolist()
train_y = train['class'].apply(classes.index).values
valid_y = valid['class'].apply(classes.index).values
test_y = test['class'].apply(classes.index).values

# Programando un perceptrón multicapa

El perceptrón multicapa consta de varias capas, al menos una de entrada y otra de salida. Vamos a programar una capa genérica usando numpy y vamos a programar el algoritmo de retropropagación usando la regla de la cadena.

## Empezando: Programando una capa y sus pesos

Primero vamos a hacer una propagación haica adelante sencilla a partir de una capa programada genéricamente. Esta capa admitirá un vector de entradas unidimensional y sacará sus respectidas salidas activadas.

In [6]:
import numpy as np

class Capa:
  def __init__(self, n_entradas, n_salidas, f_act):
      # Matriz de forma (E, S) inicializada con valores al azar de media 0 y desviacion 1
      self.pesos = np.random.randn(n_entradas, n_salidas) * 1e-2

      # Matriz de forma (S) inicializada con valores al azar de media 0 y desviacion 1
      self.bias = np.random.randn(n_salidas)
      self.f_act = f_act

  def __call__(self, entradas): # Nota: __call__ hace que puedas llamar al objeto instanciado.
      return self.f_act(entradas @ self.pesos + self.bias)
  
  def update(self, d_w, d_b):
      self.pesos += d_w
      self.bias += d_b

In [7]:
# Activacion softmax que devuelve un vector con las probabilidades que la red asigna.
def sigmoide(x):
    return 1/(1 + np.exp(-x))

# Haremos 4 entradas con 5 neuronas ocultas y 3 neuronas de salida (una para cada clase).

capa_oculta = Capa(4, 5, sigmoide) # Activacion Sigmoidal
capa_salida = Capa(5, 3, sigmoide)

def forward_propagation(entradas):
    return capa_salida(capa_oculta(entradas)) # La entrada de capa_salida es la salida de capa_oculta!

In [8]:
# Probamos con la primera fila de train. Todavía no entrenamos!
entrada_prueba = train_x[0]

# Con softmax tenemos un vector de probabilidades. La probabilidad mas alta es la predicha
predicha = forward_propagation(entrada_prueba)
predicha

array([0.50026219, 0.34334905, 0.64857804])

## Función de pérdida y retropropagación

La red necesita tener una medida de cuanto se ha equivocado. Vamos a crear una función de pérdida, en este caso Categorical Cross-Entropy, muy útil en clasificación.

In [9]:
def loss(deseada, predicha):
  deseada_cat = np.zeros(predicha.shape[0])
  deseada_cat[deseada] = 1

  return deseada_cat - predicha

In [10]:
def d_sigmoide(z):
  return z * (1-z)

def forward_back(x, y, capa_oculta, capa_salida, loss=loss, alpha=0.1):
  # Propagacion hacia delante
  s_0 = capa_oculta(x)
  s_1 = capa_salida(s_0)

  # Calculo del error
  e = loss(y, s_1)

  # Retropropagacion
  d_1 = e * d_sigmoide(s_1)
  d_0 = d_1 @ capa_salida.pesos.T * d_sigmoide(s_0)

  # Actualizacion pesos
  d_w_0 = alpha * x[:, None] @ d_0[None, :]
  d_b_0 = alpha * d_0
  d_w_1 = alpha * s_0[:, None] @ d_1[None, :]
  d_b_1 = alpha * d_1

  capa_oculta.update(d_w_0, d_b_0)
  capa_salida.update(d_w_1, d_b_1)

  return s_1.argmax(), (e**2).mean()**0.5

In [11]:
capa_oculta = Capa(4, 5, lambda x: 1/(1 + np.exp(-x))) # Activacion Sigmoidal
capa_salida = Capa(5, 3, lambda x: 1/(1 + np.exp(-x)))


In [12]:
from tqdm.auto import tqdm

# Probamos a entrenar!
epoch_e = 0
with tqdm(bar_format='e {postfix}', postfix=epoch_e) as t:
  for i in tqdm(range(10000)):
    epoch_e = 0
    for j in range(len(train_x)):
      x_ejemplo, y_ejemplo = train_x[j], train_y[j]

      salida, error = forward_back(x_ejemplo, y_ejemplo,
                                  capa_oculta=capa_oculta,
                                  capa_salida=capa_salida,
                                  loss=loss,
                                  alpha=.01)
      epoch_e+=error

    t.postfix=epoch_e/len(train_x)
    t.update()


e 

  0%|          | 0/10000 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

# Entrenamiento y evaluación

Para detener el entrenamiento vamos a usar los 15 ejemplos de validacion que hemos reservado. Como la parada en epochs es un hiperparámetro, no podemos elegir dicho hiperparámetro con test.

También vamos a elegir una métrica, en este caso vamos a usar la precisión (o Accuracy)

In [14]:
def train_with_stop_cond(data_x, data_y,
                         valid_x, valid_y,
                         c_o, c_s,
                         epochs=int(1e9),
                         l=loss,
                         delta=0.1, # Margen de parada para el error de validacion
                         a=.1):
  
  epoch_train_e, epoch_valid_e = 0, 0
  with tqdm(bar_format='valid {postfix}',
            postfix=epoch_valid_e,
            total=epochs) as t:

    last_epoch_e = 99999999
    i = 0
    while i < epochs:
      # Epoca de entrenamiento
      epoch_train_e, epoch_valid_e = 0, 0
      for j in range(len(data_x)):
        x_ejemplo, y_ejemplo = data_x[j], data_y[j]

        salida, error = forward_back(x_ejemplo, y_ejemplo,
                                    capa_oculta=c_o,
                                    capa_salida=c_s,
                                    loss=l,
                                    alpha=a)
        epoch_train_e+=error # Mucho cuidado, esto es una estimación, lo correcto es hacer otra
        # pasada sobre el conjunto de aprendizaje sin aprender (alpha=0) para calcular las métricas

      # Epoca de validación
      for j in range(len(valid_x)):
        x_ejemplo, y_ejemplo = valid_x[j], valid_y[j]

        salida, error = forward_back(x_ejemplo, y_ejemplo,
                                    capa_oculta=capa_oculta,
                                    capa_salida=capa_salida,
                                    loss=l,
                                    alpha=0) # Alpha 0 es equivalente a no actualizar los pesos!
        epoch_valid_e+=error

      t.postfix=epoch_train_e
      t.update(1)

      if epoch_valid_e > (last_epoch_e + delta):
        i = epochs + 1
      else:
        i += 1
      last_epoch_e = epoch_valid_e

In [15]:
# Reiniciamos la red 
capa_oculta = Capa(4, 5, lambda x: 1/(1 + np.exp(-x))) # Activacion Sigmoidal
capa_salida = Capa(5, 3, lambda x: 1/(1 + np.exp(-x)))

# Y ahora si, podemos reentrenar
train_with_stop_cond(train_x, train_y,
                    valid_x, valid_y,
                    capa_oculta, capa_salida,
                    epochs = 10000,
                    delta = .01, # Ajusta el alpha para ver si para demasiado pronto!
                    l=loss, a=.1)

valid 

KeyboardInterrupt: ignored

## Evaluando el modelo
Test se utiliza solo al final para obtener la precisión (Accuracy).
Vamos a comprobar que tal se le da a nuestro modelo encontrar flores.

In [16]:
def accuracy(d, p):
  return np.mean(d==p) * 100

def get_pred(entradas):
  return capa_salida(capa_oculta(entradas)).argmax()

preds = []
for i in range(len(test_x)):
    x_ejemplo, y_ejemplo = test_x[i], test_y[i]

    salida = get_pred(x_ejemplo)
    preds.append(salida)

accuracy(test_y, preds)

100.0